In [1]:
using Plotly
using DataFrames
using CSV
using Statistics

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-16610025081210000304\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-14067481520974200364\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-16610025081210000304\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [122]:
df = CSV.read("The_single_cell_transcriptome_as_a_component_system/MouseCellAtlas/mca/mainTable_Bladder.csv", DataFrame);

In [123]:
df_genes = CSV.read("MGImarkerQuery_20200914_050053.txt", DataFrame);

┌ Warning: thread = 1 warning: only found 10 / 11 columns around data row: 2. Filling remaining columns with `missing`
└ @ CSV /opt/julia/packages/CSV/la2cd/src/file.jl:603
┌ Warning: thread = 1 warning: only found 10 / 11 columns around data row: 3. Filling remaining columns with `missing`
└ @ CSV /opt/julia/packages/CSV/la2cd/src/file.jl:603
┌ Warning: thread = 1 warning: only found 10 / 11 columns around data row: 4. Filling remaining columns with `missing`
└ @ CSV /opt/julia/packages/CSV/la2cd/src/file.jl:603
┌ Warning: thread = 1 warning: only found 10 / 11 columns around data row: 5. Filling remaining columns with `missing`
└ @ CSV /opt/julia/packages/CSV/la2cd/src/file.jl:603
┌ Warning: thread = 1 warning: only found 10 / 11 columns around data row: 6. Filling remaining columns with `missing`
└ @ CSV /opt/julia/packages/CSV/la2cd/src/file.jl:603
┌ Warning: thread = 1 warning: only found 10 / 11 columns around data row: 7. Filling remaining columns with `missing`
└ @ CSV /opt/jul

In [124]:
unique(df_genes[!,"Feature Type"])

23-element Array{String,1}:
 "RNase MRP RNA gene"
 "telomerase RNA gene"
 "scRNA gene"
 "non-coding RNA gene"
 "sense overlapping lncRNA gene"
 "RNase P RNA gene"
 "bidirectional promoter lncRNA gene"
 "sense intronic lncRNA gene"
 "SRP RNA gene"
 "ribozyme gene"
 "lincRNA gene"
 "antisense lncRNA gene"
 "rRNA gene"
 "tRNA gene"
 "protein coding gene"
 "gene segment"
 "unclassified non-coding RNA gene"
 "snRNA gene"
 "snoRNA gene"
 "miRNA gene"
 "heritable phenotypic marker"
 "unclassified gene"
 "lncRNA gene"

In [125]:
lncgenes = df_genes[df_genes[!,"Feature Type"].=="lncRNA gene",:Symbol];

In [126]:
df_lnc = df[[g in lncgenes for g in df.Column1], :];

In [127]:
f_lnc = []
M = [sum(col) for col in eachcol(df_lnc)[2:end]];
for row in eachrow(df_lnc)[2:end]
    append!(f_lnc, reduce(+, [x/m for (x,m) in zip(row[2:end],M) if m>0], dims=1)) 
end

In [128]:
f = []
M = [sum(col) for col in eachcol(df)[2:end]];
for row in eachrow(df)[2:end]
    append!(f, reduce(+, [x/m for (x,m) in zip(row[2:end],M) if m>0], dims=1)) 
end

In [129]:
f = sort(f ./ sum(f), rev=true);
f_lnc = sort(f_lnc ./ sum(f_lnc), rev=true);

In [130]:
sum(skipmissing([x for x in row[2:end]]))

5.0

In [131]:
trace = Plotly.scatter(y=f, name="data", line_color="blue", line_width=10)
tracelnc = Plotly.scatter(y=f_lnc, name="lnc", line_color="purple", line_width=10)


layout = Layout(
    title = "Lung MCA",
    xaxis_type="log",
    xaxis_tickfont_size=20,
    xaxis_title = "rank, i",
    xaxis_titlefont = 25,
    #xaxis_tickformat = "e",
    yaxis_type="log",
    yaxis_tickformat = "e",
    yaxis_title = "frequency, fi",
    yaxis_titlefont = 25,
    yaxis_tickfont_size=20,
    legendfont_size=30
)


data = [trace, tracelnc]

p = Plotly.plot(data, layout)
Plotly.savefig(p, "zipf_lnc_bladder.pdf")

"zipf_lnc_bladder.pdf"

In [ ]:
cv2lnc = [var(row) / mean(row) / mean(row)  for row in eachrow(df_lnc[2:end,2:end])];
meanslnc = [mean(row)  for row in eachrow(df_lnc[2:end,2:end])];

cv2 = [var(row) / mean(row) / mean(row)  for row in eachrow(df[2:end,2:end])];
means = [mean(row)  for row in eachrow(df[2:end,2:end])];

In [ ]:
trace = Plotly.scatter(x=means, y=cv2, name="data", line_color="gray", mode="markers", line_width=10)
tracelnc = Plotly.scatter(x=meanslnc, y=cv2lnc, name="lnc", line_color="purple", mode="markers", line_width=10)


layout = Layout(
    title = "Lung MCA",
    xaxis_tickfont_size=20,
     xaxis_type="log",
    xaxis_title = "mean",
    xaxis_titlefont = 25,
    yaxis_type="log",
    yaxis_tickformat = "e",
    yaxis_title = "CV2",
    yaxis_titlefont = 25,
    yaxis_tickfont_size=20,
    legendfont_size=30
)


data = [trace, tracelnc]

p = Plotly.plot(data, layout)
Plotly.savefig(p, "cv2_lnc_bladder.pdf")